## Contexto
Uma empresa lançou um teste A/B para avaliar a usabilidade do menu lateral de sua plataforma. O teste compara duas configurações:
1. Acesso direto a uma página específica do site (Nível 0 do menu lateral).
2. Acesso à mesma página, mas sob uma aba mais geral (Nível 1 do menu lateral).
O objetivo do desafio a seguir é determinar qual configuração do menu lateral leva a mais acessos e conversões para a página
específica da plataforma . Identificar a configuração mais eficaz ajudará a otimizar a navegação do site para aumentar o engajamento
dos usuários.

## Base de Dados
A base de dados anexada a este desafio inclui as seguintes colunas:

`user_id` : identificador único do usuário

`device` : dispositivo do qual o usuário acessou a plataforma

`country` : país do qual o usuário acessou a plataforma

`timestamp` : data e horário em que o usuário acessou a plataforma

`group` : versão do menu lateral (Nível 0 ou Nível 1)

`converted` : conversão do usuário para a página de interesse

`session_duration` : tempo (s) em que o usuário permaneceu ativo na página de interesse

## Questões
1. Qual configuração do menu lateral levou mais usuários a acessarem a página específica?
2. Qual o tempo médio de sessão para cada configuração do menu?
3. Qual é a taxa de conversão para cada configuração do menu?
4. A diferença nos acessos entre as configurações é estatisticamente significativa?
5. Quais insights podem ser derivados dos dados para otimizar a navegação do site e aumentar o engajamento dos usuários?
6. Existe alguma tendência temporal (comportamento ao longo dia) nos acessos ou conversões para cada configuração do menu?
7. Quais recomendações você faria com base nos dados analisados para otimizar a navegação do site?




In [1]:
# importando as biblioteccas
import pandas as pd

In [2]:
# abrindo df
df = pd.read_excel(r'C:\Users\earap\OneDrive\Desktop\Dynamox\ab_test_data_1.xlsx')

In [3]:
# escrevendo os 10 primeiros registros do df
df.head(10)

,user_id,device,country,timestamp,group,converted,session_duration,hour_of_day
0,42,Smartphone,França,2024-06-01 06:02:00,Nível 0,1,18.675765,6
1,1650,Tablet,Reino Unido,2024-06-01 14:41:00,Nível 0,1,4.283270,14
2,1308,Tablet,Reino Unido,2024-06-01 08:19:00,Nível 1,1,17.949853,8
3,671,Smartphone,França,2024-06-01 15:29:00,Nível 0,1,18.844162,15
4,51,Smartphone,Brasil,2024-06-01 14:30:00,Nível 0,1,4.285244,14
5,579,Desktop,França,2024-06-01 11:25:00,Nível 1,0,0.000000,11
6,852,Desktop,Canadá,2024-06-01 18:08:00,Nível 0,0,0.000000,18
7,61,Desktop,Canadá,2024-06-01 08:13:00,Nível 0,0,0.000000,8
8,1537,Tablet,Brasil,2024-06-01 19:30:00,Nível 0,0,0.000000,19
9,1431,Smartphone,Índia,2024-06-01 13:29:00,Nível 1,0,0.000000,13


In [4]:
# analisando o resumo estatístico dos dados
df.describe()

,user_id,timestamp,converted,session_duration,hour_of_day
count,2305.000000,2305,2305.000000,2269.000000,2305.000000
mean,1005.528850,2024-06-01 12:04:40.789587712,0.350976,5.976675,11.587419
min,1.000000,2024-06-01 00:00:00,0.000000,0.000000,0.000000
25%,540.000000,2024-06-01 07:04:00,0.000000,0.000000,7.000000
50%,1022.000000,2024-06-01 11:57:00,0.000000,0.000000,11.000000
75%,1470.000000,2024-06-01 17:17:00,1.000000,4.110315,17.000000
max,1999.000000,2024-06-01 23:58:00,1.000000,64.446394,23.000000
std,567.135116,NaN,0.477379,12.414010,6.474329


In [5]:
# escrevendo o número de linhas e colunas
df.shape

(2305, 8)

### Tratamento de nulos e duplicados

Como visto nas estatísticas acima, há a ocorrência de valores nulos para as variáveis. Para isso, vou entender a distribuição e fazer sua correção.

In [6]:
# somando os valores nulos de cada variável
df.isnull().sum()

user_id              0
device               0
country              0
timestamp            0
group                0
converted            0
session_duration    36
hour_of_day          0
dtype: int64

In [7]:
# dropando valores nulos
df_no_nan = df.dropna()
df_no_nan.shape

(2269, 8)

In [8]:
# escrevendo as linhas duplicadas.
df_valor_duplicado = df_no_nan.duplicated().sum()
df_valor_duplicado 

96

In [9]:
# removendo as linhas duplicadas
df_final = df_no_nan.drop_duplicates()
df_final

,user_id,device,country,timestamp,group,converted,session_duration,hour_of_day
0,42,Smartphone,França,2024-06-01 06:02:00,Nível 0,1,18.675765,6
1,1650,Tablet,Reino Unido,2024-06-01 14:41:00,Nível 0,1,4.283270,14
2,1308,Tablet,Reino Unido,2024-06-01 08:19:00,Nível 1,1,17.949853,8
3,671,Smartphone,França,2024-06-01 15:29:00,Nível 0,1,18.844162,15
4,51,Smartphone,Brasil,2024-06-01 14:30:00,Nível 0,1,4.285244,14
...,...,...,...,...,...,...,...,...
2300,1587,Desktop,Índia,2024-06-01 05:25:00,Nível 0,0,0.000000,5
2301,838,Tablet,Brasil,2024-06-01 07:14:00,Nível 1,1,2.292511,7
2302,1823,Tablet,Índia,2024-06-01 09:09:00,Nível 1,0,0.000000,9
2303,1853,Desktop,Austrália,2024-06-01 07:19:00,Nível 1,1,1.908920,7


In [10]:
df_final.to_csv(r'C:\Users\earap\OneDrive\Desktop\Dynamox\df_final.csv', index=False, encoding='utf-8')